<a href="https://colab.research.google.com/github/MohabSaber/Constant_Area_Coding/blob/main/Constant_Area_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deap


In [ ]:
import heapq
from collections import defaultdict
import numpy as np
import random
from deap import base, creator, tools, algorithms


#huffman encoding to compress data
def huffman_encoding(data):
    frequency = defaultdict(int)
    for symbol in data:
        frequency[symbol] += 1

    heap = [[weight, [symbol, ""]] for symbol, weight in frequency.items()]
    heapq.heapify(heap)

#build huffman tree
    while len(heap) > 1:
        lo = heapq.heappop(heap)
        hi = heapq.heappop(heap)
        for pair in lo[1:]:
            pair[1] = '0' + pair[1]
        for pair in hi[1:]:
            pair[1] = '1' + pair[1]
        heapq.heappush(heap, [lo[0] + hi[0]] + lo[1:] + hi[1:])
#generate huffman code
    huff_dict = {pair[0]: pair[1] for sublist in heap[0][1:] for pair in [sublist]} #dictionary of huffman
    encoded_data = ''.join(huff_dict[symbol] for symbol in data)

    return encoded_data, huff_dict

#applies huffman encoding to data
def huffman_compression(data, block_size):
    flat_data = data.flatten()
    encoded_data, huff_dict = huffman_encoding(flat_data)
    original_size = len(flat_data) * 8  # element = 8 bits
    compressed_size = len(encoded_data)  # bits

    CR = original_size / compressed_size
    RD = 1 - (1 / CR)

    return CR, RD

def fitness(individual, data):
    block_size = tuple(individual)
    CR, RD = huffman_compression(data, block_size)
    return CR,


#perform GA starting from population intializing till selecting next generation
data = np.random.randint(0, 2, size=(100, 100), dtype=np.uint8)



creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 1, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_int, 2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness, data=data)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutUniformInt, low=1, up=10, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

#number of generations (100), crossover probability (0.7), and mutation probability (0.3).
def main():
    population = toolbox.population(n=200)
    ngen = 100
    cxpb = 0.7
    mutpb = 0.3
# we collect statistics for GA to evaluate performance and watch the progress

#hall of fame store best indvidual found during evolution process
#then we use best indvidual to get compression performance
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
#to see how fitness of population changes over generations
    algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, stats=stats, halloffame=hof)

    return population, stats, hof

if __name__ == "__main__":
    pop, stats, hof = main()
    best_individual = hof[0]
    best_block_size = tuple(best_individual)
    best_CR, best_RD = huffman_compression(data, best_block_size)

    print(f"Best Block Size: {best_block_size}")
    print(f"Compression Ratio (CR): {best_CR}")
    print(f"Relative Data Redundancy (RD): {best_RD}")


gen	nevals	avg	std	min	max
0  	200   	8  	0  	8  	8  
1  	155   	8  	0  	8  	8  
2  	159   	8  	0  	8  	8  
3  	157   	8  	0  	8  	8  
4  	165   	8  	0  	8  	8  
5  	142   	8  	0  	8  	8  
6  	162   	8  	0  	8  	8  
7  	171   	8  	0  	8  	8  
8  	154   	8  	0  	8  	8  
9  	148   	8  	0  	8  	8  
10 	156   	8  	0  	8  	8  
11 	156   	8  	0  	8  	8  
12 	159   	8  	0  	8  	8  
13 	152   	8  	0  	8  	8  
14 	160   	8  	0  	8  	8  
15 	163   	8  	0  	8  	8  
16 	160   	8  	0  	8  	8  
17 	164   	8  	0  	8  	8  
18 	157   	8  	0  	8  	8  
19 	153   	8  	0  	8  	8  
20 	163   	8  	0  	8  	8  
21 	162   	8  	0  	8  	8  
22 	157   	8  	0  	8  	8  
23 	159   	8  	0  	8  	8  
24 	155   	8  	0  	8  	8  
25 	148   	8  	0  	8  	8  
26 	167   	8  	0  	8  	8  
27 	161   	8  	0  	8  	8  
28 	168   	8  	0  	8  	8  
29 	165   	8  	0  	8  	8  
30 	156   	8  	0  	8  	8  
31 	162   	8  	0  	8  	8  
32 	175   	8  	0  	8  	8  
33 	159   	8  	0  	8  	8  
34 	152   	8  	0  	8  	8  
35 	165   	8  	0  	8  	8  
3